In [15]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to perform Google search and get links
def get_search_results(query, num_results=2):
    search_results = search(query, num_results=num_results)
    return list(search_results)

# Function to extract content from a search result
def extract_content_from_url(url, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Extract all text content from the page
            return soup.get_text(separator="\n")
        else:
            return f"Failed to retrieve webpage. Status code: {response.status_code}"
    except requests.exceptions.Timeout:
        return "Request timed out"
    except requests.exceptions.RequestException as e:
        return f"Request failed: {e}"

# Function to search for specific terms in the content
def search_for_terms(content, terms):
    for term in terms:
        if term.lower() in content.lower():
            return True
    return False

# Read symbols and dates from the CSV file
input_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/TestData.csv'
df_symbols = pd.read_csv(input_file_path)
symbols = df_symbols['Symbol'].tolist()
dates = df_symbols['Date'].tolist()

# Terms to search for
terms = ["round", "rounded", "rounding", "rounds"]

# Prepare the result list
results = []

# Process each symbol and its corresponding date
for symbol, date in zip(symbols, dates):
    query = f"{symbol} Reverse split press release"
    print(f"\nSearching for: {query}")

    # Get the top 2 search results
    links = get_search_results(query, num_results=2)
    
    # Flag to track if "DO BUY" should be set
    do_buy_flag = False

    # Extract and check content from the search results
    for i, link in enumerate(links):
        print(f"\nExtracting content from search result {i+1}: {link}")
        content = extract_content_from_url(link)
        
        # Check for specific terms in the content
        if search_for_terms(content, terms):
            do_buy_flag = True
            break

    # Determine final decision
    decision = "ROUNDING" if do_buy_flag else "FRACTIONAL"
    results.append([date, symbol, decision])
    print(f"Decision for {symbol}: {decision}")

# Create a DataFrame from the results
result_df = pd.DataFrame(results, columns=['Date', 'Symbol', 'Decision'])

# Save the results to a new CSV file
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/Results.csv'
result_df.to_csv(output_file_path, index=False)

print(f"\nResults have been saved to {output_file_path}")



Searching for: ALTAF Reverse split press release

Extracting content from search result 1: https://investors.atarabio.com/news-events/press-releases/detail/352/atara-biotherapeutics-announces-1-for-25-reverse-stock-split

Extracting content from search result 2: https://www.marketwatch.com/investing/stock/altaf

Extracting content from search result 3: https://ir.crownek.com/news-events/press-releases/detail/123/crown-electrokinetics-announces-1-for-150-reverse-stock
Decision for ALTAF: ROUNDING

Searching for: MDRR Reverse split press release

Extracting content from search result 1: https://www.businesswire.com/news/home/20240620079723/en/Medalist-Diversified-REIT-Inc.-Announces-Reverse-Stock-Split-and-Forward-Stock-Split

Extracting content from search result 2: https://www.nasdaq.com/press-release/medalist-diversified-reit-inc-announces-reverse-stock-split-and-forward-stock-split

Extracting content from search result 3: https://medalistreit.com/investors/newsroom/

Extracting con